# This notebook will be used for capstone project

In [50]:
import pandas as pd
import numpy as np
import bs4 as bs
import requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')
print ("Hello Capstone Project Course!")

Libraries imported.
Hello Capstone Project Course!


In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

dfs = pd.read_html(url, header=0)
toronto_df = dfs[0]

toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping the rows where Borough is Not assigned


In [15]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']


In [16]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Functions for processing of data¶


In [18]:
def concat_neighborhoods(series):
    string = ''
    for item in series:
        string = string + item + ', '
    return string

def ret_unique(series):
    items = []
    for item in series:
        items.append(item.split(',')[0])
    return items

def set_neighborhood(borough, neighborhood):
    items = []
    for boro, item in zip(borough, neighborhood):
        if item == 'Not assigned':
            items.append(boro)
        elif item:
            items.append(item)
            
    return items

In [19]:
toronto_aggregated = toronto_df.groupby(toronto_df['Postcode']).aggregate(concat_neighborhoods)
toronto_aggregated = toronto_aggregated.reset_index()
toronto_aggregated['Neighbourhood'] = toronto_aggregated['Neighbourhood'].astype(str).str[:-2]
toronto_aggregated['Borough'] = ret_unique(toronto_aggregated['Borough'])
toronto_aggregated.columns = ['PostCode', 'Borough', 'Neighborhood']
toronto_aggregated['Neighborhood'] = set_neighborhood(toronto_aggregated['Borough'], toronto_aggregated['Neighborhood'])
toronto_aggregated.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
print("The shape of Cleansed Toronto Dataframe is : ", toronto_aggregated.shape)


The shape of Cleansed Toronto Dataframe is :  (103, 3)


## Adding coordinates of each individual Neighborhood using geocoder package¶


In [38]:
from geopy.geocoders import Nominatim 
 

In [43]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
toronto_merged = toronto_aggregated.join(geodata.set_index('Postal Code'), on='PostCode')
toronto_merged.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [51]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


# Visualize the Neighborhoods¶


In [58]:
import folium

ModuleNotFoundError: No module named 'folium'

In [59]:
toronto_map = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

for lat, lng, neighborhood in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color= 'red', 
    fill = True,
    fill_color = '#3186cc', 
    fill_opacity = 0.5,
    parse_html = False).add_to(toronto_map)

toronto_map

NameError: name 'folium' is not defined